# Steganografia - testy narzędzi
Najnowsza wersja sprawozdania jest dostępna pod adresem: [https://github.com/Gombek7/techniki-poufnosci/blob/main/Steganografia%20-%20testy%20narz%C4%99dzi/sprawozdanie.ipynb](https://github.com/Gombek7/techniki-poufnosci/blob/main/Steganografia%20-%20testy%20narz%C4%99dzi/sprawozdanie.ipynb)

Członkowie zespołu:
- Jarosław Dakowicz
- Piotr Kozioł
- Anton Maisiuk

## Instalacja środowiska

Sprawozdanie zostało napisane w notatniku Jupyter z użyciem jądra Deno, które pozwala na wykonywanie kodu w języku TypeScript zamiast w Pythonie.

Najpierw neleży zainstalować Deno. Należy uruchomić poniższą komendę w powershell.

```powershell
irm https://deno.land/install.ps1 | iex
```

Następnie, aby zainstalować jądro, należy użyć poniższej komendy.
```powershell
deno jupyter --install
```

Od teraz podczas edycji notatnika Jupyter w Visual Studio Code można wybrać jądro Deno.

## Zadanie 1 - Ocena narzędzi steganograficznych 

Na podstawie listy ze strony [Wikipedii](https://en.wikipedia.org/wiki/Steganography_tools) wybierz 5 narzędzi steganograficznych i dokonaj ich oceny (badań):
- porównaj szybkość działania dla tego samego pliku,
- jeżeli to możliwe, to oceń ilość/jakość zmian pliku źródłowego.


## Zadanie 2 - Własny system steganograficzny

Wykonaj prosty system steganograficzny plików graficznych BMP polegający na osadzaniu w najmniej znaczących bitach danego stegotekstu:
- ilość bitów ma być zmienna, od 1 do 4,
- dokonaj oceny wizualnej głębokości zmian pliku źródłowego w zależności od ilości bitów modyfikowanych.

In [30]:
0b11100000 >> 5

7

In [7]:
function* getBitsIterator(bytes: Uint8Array, bits: 1 | 2 | 3 | 4) {
    switch (bits) {
        case 1:
            for (const byte of bytes) {
                yield (byte & 0b10000000) >> 7;
                yield (byte & 0b01000000) >> 6;
                yield (byte & 0b00100000) >> 5;
                yield (byte & 0b00010000) >> 4;
                yield (byte & 0b00001000) >> 3;
                yield (byte & 0b00000100) >> 2;
                yield (byte & 0b00000010) >> 1;
                yield (byte & 0b00000001);
            }
            break;
        case 2:
            for (const byte of bytes) {
                yield (byte & 0b11000000) >> 6;
                yield (byte & 0b00110000) >> 4;
                yield (byte & 0b00001100) >> 2;
                yield (byte & 0b00000011);
            }
            break;
        case 3:
            {
                let buffer = 0; // Holds the current bits buffer
                let bufferLength = 0; // Number of bits available in the buffer

                for (const byte of bytes) {
                    buffer = (buffer << 8) | byte; // Add new byte to the buffer
                    bufferLength += 8; // Update buffer length

                    while (bufferLength >= 3) { // Emit as long as we can get 3 bits
                        yield (buffer >> (bufferLength - 3)) & 0b111; // Extract top 3 bits
                        bufferLength -= 3; // Reduce buffer length
                    }
                }

                // Handle leftover bits (if any) by padding with zeros
                if (bufferLength > 0) {
                    yield (buffer << (3 - bufferLength)) & 0b111; // Pad with zeros
                }
            }
            break;
        case 4:
            for (const byte of bytes) {
                yield (byte & 0b11110000) >> 4;
                yield (byte & 0b00001111);
            }
            break;
    }
}

0
1
0
1
0
1
0
1


In [45]:
//@deno-types="npm:@types/bmp-js"
import { decode, encode } from "npm:bmp-js";
import fs from "node:fs";
import type { Buffer } from "node:buffer";

function embedText(inputFile: Buffer, text: string, bits: 1 | 2 | 3 | 4) {
    const bmpData = decode(inputFile);
    const textEncoder = new TextEncoder();
    const textBytes = textEncoder.encode(text + "\0");
    const bitsIterator: Generator<number, void, unknown> = getBitsIterator(
        textBytes,
        bits,
    );
    let i = 0;
    const mask = 255 << bits;
    for (const chunk of bitsIterator) {
        // don't write to alpha channel
        if (!(i % 4)) i++;

        bmpData.data[i] = (bmpData.data[i] & mask) | chunk;
        console.log(bmpData.data[i]);
        i++;
    }
    console.log("---");
    return encode(bmpData);
}

const bmpBuffer = fs.readFileSync("cat.bmp");
const bmpModified = embedText(bmpBuffer, "To jest jakiś dłuższy tekst. Czy widać różnicę?", 2);

fs.writeFileSync("catModified.bmp", bmpModified.data);

function readEmbeddedText(inputFile: Buffer, bits: 1 | 2 | 3 | 4) {
    const bmpData = decode(inputFile);
    const mask = 255 >> (8 - bits);

    const readBytes: number[] = [];
    let chunkBuffer = 0;
    let chunkLeftLength = 8;
    let i = -1;
    for (const byte of bmpData.data) {
        // don't read from alpha channel
        if (!(++i % 4)) continue; 
        
        console.log(byte);
        const chunk = byte & mask;

        chunkBuffer = chunkBuffer << bits;
        chunkBuffer = chunkBuffer | chunk;
        chunkLeftLength -= bits;

        if (chunkLeftLength === 0) {
            readBytes.push(chunkBuffer);
            // end if found '\0' character
            if (chunkBuffer === 0) {
                break;
            }
            chunkBuffer = 0;
            chunkLeftLength = 8;
        }
    }

    const textDecoder = new TextDecoder();
    return textDecoder.decode(new Uint8Array(readBytes)).slice(0, -1);
}

const readFile = fs.readFileSync("catModified.bmp");
readEmbeddedText(readFile, 2);


53
45
53
48
45
54
59
47
60
58
44
60
57
46
62
58
45
62
57
45
61
63
52
67
57
51
61
56
48
62
64
52
65
66
50
66
65
54
64
61
49
62
66
51
65
62
46
61
63
48
61
61
50
65
62
51
60
62
48
60
69
54
65
60
47
60
69
53
66
68
52
66
73
59
69
73
59
64
77
61
70
83
67
68
85
67
72
87
69
75
90
74
77
95
78
77
88
74
76
92
77
83
89
72
77
94
77
77
93
78
78
91
73
79
92
79
81
91
73
76
88
74
79
98
80
82
96
80
81
100
84
87
101
91
86
106
89
91
106
93
88
110
92
92
109
99
93
115
97
94
114
101
93
114
97
92
113
102
96
113
103
96
117
104
98
116
105
99
116
106
100
116
105
103
116
106
103
120
108
103
122
111
100
123
111
104
125
113
106
131
119
108
133
126
119
138
129
122
146
133
125
146
132
127
143
132
125
148
138
133
154
141
132
159
147
135
160
148
132
160
---
53
45
53
48
45
54
59
47
60
58
44
60
57
46
62
58
45
62
57
45
61
63
52
67
57
51
61
56
48
62
64
52
65
66
50
66
65
54
64
61
49
62
66
51
65
62
46
61
63
48
61
61
50
65
62
51
60
62
48
60
69
54
65
60
47
60
69
53
66
68
52
66
73
59
69
73
59
64
77
61
70
83
67
68
85
67
72
87
69

"To jest jakiś dłuższy tekst. Czy widać różnicę?"